<a href="https://colab.research.google.com/github/shpotes/tensorflowers/blob/resnet-baseline/notebooks/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/
!rm -rf tensorflowers
!git clone -b resnet-baseline https://github.com/shpotes/tensorflowers.git
%cd tensorflowers
!pip install -qq -r requirements.txt

In [11]:
import torch.nn as nn
from torchvision import models
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

from src.dataset import TFColDataModule
from src.models.pl_module import HydraModule

In [12]:
dm = TFColDataModule(
    image_transforms=transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
)

In [13]:
base_backbone = models.resnet18(pretrained=True)
backbone = nn.Sequential(*list(base_backbone.children())[:-1])

for param in backbone.parameters():
    param.requires_grad = False

model = HydraModule(
    backbone,
    lr=3e-4,
)

In [17]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [18]:
logger = WandbLogger(
    project="challenge", 
    name="baseline",
    entity="tensorflowers",
)
trainer = pl.Trainer(
    max_epochs=10,
    gpus=1,
    logger=logger    
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model, dm)

Downloading:   0%|          | 0.00/4.08k [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/687M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/469k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/225M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/155k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]